In [7]:
!pip install statsmodels
!pip install prophet
!pip install u8darts

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 610.0 kB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 934.4/934.4 kB 6.8 MB/s eta 0:00:00:00:01


In [8]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [320]:
import pandas as pd
import matplotlib.pyplot as plt
import utils.helpers as helpers
from darts import TimeSeries
from darts.models import RNNModel, AutoARIMA
from sklearn.preprocessing import StandardScaler

In [271]:
bronx = pd.read_parquet('../data/Bronx - all txns.parquet.gz')
bronx.index = pd.to_datetime(bronx.index)
#bronx = bronx.reset_index()
#bronx['timestamp_hour'] = pd.to_datetime(bronx['timestamp_hour'])
bronx = bronx.sort_values(by = 'timestamp_hour')
bronx = bronx.rename(columns = {'num_txns_All': 'Rides', 'timestamp_hour': 'DateTime'})
bronx.head()

,txn_date,quarter,month,day_of_month,txn_hour,week_day,is_weekend,is_monday,is_friday,is_holiday,...,lag_139,lag_140,lag_141,lag_142,lag_143,lag_145,rolling_avg_24_48_72_lags,std_24_48_72_lags,rolling_avg_w1-w4_lags,std_w1-w4_lags
timestamp_hour,,,,,,,,,,,,,,,,,,,,,
2019-02-01 00:00:00,2019-02-01,1,2,1,0,5,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-01 01:00:00,2019-02-01,1,2,1,1,5,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-01 02:00:00,2019-02-01,1,2,1,2,5,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-01 03:00:00,2019-02-01,1,2,1,3,5,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-02-01 04:00:00,2019-02-01,1,2,1,4,5,1,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [272]:
target_cols = ['Rides']
covariates_cols = ['month', 'day_of_month', 'txn_hour', 'week_day',
       'is_weekend', 'is_monday', 'is_friday', 'is_holiday',
       'is_holiday_next_day', 'is_holiday_previous_day', 'is_long_weekend',
       'is_rush_hour', 'is_business_hour', 'is_night_hour', 'sin_hour',
       'cos_hour', 'rolling_avg_w1-w4_lags']
columns = target_cols + covariates_cols

In [273]:
bronx = bronx.asfreq('h')
bronx = bronx.fillna(method='ffill')  # Forward fill to handle missing values

/tmp/ipykernel_18/1061286425.py:2: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  bronx = bronx.fillna(method='ffill')  # Forward fill to handle missing values


In [274]:
bronx.shape

(48936, 69)

In [275]:
bronx = bronx.drop(['txn_date'], axis = 1)
X = bronx[covariates_cols]
y = bronx[target_cols]

In [285]:
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X), columns = covariates_cols, index = X.index)
bronx_scaled = pd.concat([y, X_scaled], axis = 1)

In [286]:
bronx_scaled = TimeSeries.from_dataframe(bronx_scaled)

In [287]:
#bronx_scaled = bronx_scaled[pd.to_datetime('2024-01-01'):]
bronx_hist = bronx_scaled[pd.to_datetime('2024-01-01'):pd.to_datetime('2024-07-31 23:59:59')]
bronx_test = bronx_scaled[pd.to_datetime('2024-08-01'):]
bronx_hist.shape, bronx_test.shape

((5112, 18, 1), (744, 18, 1))

In [315]:
model = RNNModel(
    model="LSTM",  # Specify LSTM as the RNN type
    input_chunk_length= 24,
    training_length= 48,
    hidden_dim= 3,
    dropout = 0,
    batch_size = 50,
    n_rnn_layers= 3,
    random_state = 42,
    n_epochs = 10,
    log_tensorboard=True
)

In [ ]:
model="LSTM",
    hidden_dim=20,
    dropout=0,
    batch_size=16,
    n_epochs=300,
    optimizer_kwargs={"lr": 1e-3},
    model_name="Air_RNN",
    log_tensorboard=True,
    random_state=42,
    training_length=20,
    input_chunk_length=14,
    force_reset=True,
    save_checkpoints=True,

In [308]:
model.fit(bronx_hist[target_cols], future_covariates=bronx_scaled[covariates_cols])
forecast = model.predict(len(bronx_test), future_covariates=bronx_scaled[covariates_cols])

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 9.9 K  | train
6 | V               | Linear           | 21     | train
-------------------------------------------------------------
9.9 K     Trainable params
0         Non-trainable params
9.9 K     Total params
0.040     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [316]:
model.fit(bronx_hist[target_cols])
forecast = model.predict(len(bronx_test))

GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs

  | Name            | Type             | Params | Mode 
-------------------------------------------------------------
0 | criterion       | MSELoss          | 0      | train
1 | train_criterion | MSELoss          | 0      | train
2 | val_criterion   | MSELoss          | 0      | train
3 | train_metrics   | MetricCollection | 0      | train
4 | val_metrics     | MetricCollection | 0      | train
5 | rnn             | LSTM             | 264    | train
6 | V               | Linear           | 4      | train
-------------------------------------------------------------
268       Trainable params
0         Non-trainable params
268       Total params
0.001     Total estimated model params size (MB)
7         Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=10` reached.
GPU available: False, used: False
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Predicting: |          | 0/? [00:00<?, ?it/s]

In [309]:
__ = helpers.performance_metrics(bronx_test['Rides'].values(),forecast.values())

MAE: 3252.00
MAPE: 97.32%
RMSE: 3508.71
R2: -6.09


In [ ]:
forecast

In [318]:
len(bronx_test)

744

In [323]:
model = AutoARIMA(start_p=1, max_p=24, start_q=1)
model.fit(bronx_hist['Rides'])
forecast = model.predict(len(bronx_test))

In [325]:
__ = helpers.performance_metrics(bronx_test['Rides'].values(),forecast.values())

MAE: 1030.76
MAPE: 55.62%
RMSE: 1312.32
R2: 0.01
